In [1]:
from Process_Classes import  information
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report ,r2_score, mean_squared_error
from sklearn.metrics import roc_auc_score,roc_curve,f1_score
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier

from warnings import filterwarnings

filterwarnings('ignore')

In [ ]:
df = information("data/","hmelq.csv")

In [ ]:
df.get_inf(True,"drop")

In [ ]:
df.cattodummy()

In [ ]:
y,X= df.choose_your_y("bad")

In [ ]:
X = X.drop("reason_HomeImp",axis=1)

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size = 0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
cart = DecisionTreeClassifier()
cart_model = cart.fit(X_train,y_train)
cart_model

In [ ]:
y_pred = cart_model.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
cart_grdi = {"max_depth": range(1,10),
            "min_samples_split": list(range(2,50)),
            "criterion":['gini', 'entropy']}

In [ ]:
cart = tree.DecisionTreeClassifier()
cart_cv = GridSearchCV(cart, cart_grdi, cv=10, n_jobs=-1, verbose=2,scoring='f1')
cart_cv_model = cart_cv.fit(X_train,y_train)

In [ ]:
print("En iyi parametlerler: " + str(cart_cv_model.best_params_))

In [ ]:
cart_cv_model.best_params_['criterion']

In [ ]:
b = {for i in ['criterion','max_depth','min_samples_split'] i:cart_cv_model.best_params_[i]}

In [ ]:
st = ''
for i in cart_cv_model.best_params_:
    st = st + str(str(i) +'='+ '"'+str(cart_cv_model.best_params_[i])+'"'+ ',')
st


In [ ]:
[i+' =' + str(cart_cv_model.best_params_[i]) for i in cart_cv_model.best_params_]

In [ ]:
cart = tree.DecisionTreeClassifier(criterion ='entropy', max_depth =8, min_samples_split =4)
cart_tuned = cart.fit(X_train,y_train)

In [ ]:
cart = tree.DecisionTreeClassifier(criterion='entropy',max_depth = 8, min_samples_split=2)
cart_tuned = cart.fit(X_train,y_train)

In [ ]:
y_pred = cart_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier().fit(X_train,y_train)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
rf_params = {"max_depth": [8,10,11,13,15,18],
            "max_features": [5,10,15,20],
             "n_estimators": [5,10,50,100,200,500],
             "min_samples_split": [3,5,10],
            "criterion":['entropy']}

In [ ]:
rf_model = RandomForestClassifier()
rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10, n_jobs=-1,verbose=2,scoring='f1')

In [ ]:
rf_cv_model.fit(X_train,y_train)

In [ ]:
rf_cv_model.best_params_

In [ ]:
rf_tuned = RandomForestClassifier(max_depth=13,max_features=15,min_samples_split=10, n_estimators=10,criterion='entropy')

In [ ]:
rf_tuned.fit(X_train,y_train)
y_pred = rf_tuned.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
importance = pd.DataFrame({"Importance": rf_tuned.feature_importances_*100},index=X_train.columns)
importance.sort_values(by= "Importance", axis=0,ascending=True)[0:20].plot(kind="barh")

In [ ]:
good_cat = list(importance.sort_values(by= "Importance", axis=0,ascending=False)[0:10].T.columns)

In [ ]:
X = X[good_cat]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size = 0.3, random_state=42, shuffle=False)

In [ ]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(100,20)).fit(X_train_scaled,y_train)
y_pred = mlp_model.predict(X_test_scaled)
print(classification_report(y_test,y_pred))

In [ ]:
mlpc_params = {
    'alpha':[0.1,0.01,0.001,0.005,0.0001,0.00001],
    'hidden_layer_sizes': [(10,10,10),(45,50,60),(25,35,45),(15,15)],
    'solver': ['lbfgs','adam','sgd'],
    'activation': ['relu','logistic','tanh','identity']  
}

In [ ]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc,mlpc_params,cv =10 , n_jobs=-1,verbose = 2,scoring='f1_weighted')
mlpc_cv_model.fit(X_train_scaled,y_train)

In [ ]:
mlpc_cv_model.best_params_

In [ ]:
mlpc_tuned = MLPClassifier(activation='relu',alpha=0.1,hidden_layer_sizes= (10,10,10),solver='lbfgs')

In [ ]:
mlpc_tuned.fit(X_train_scaled,y_train)
y_pred=mlpc_tuned.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
y,X= df.choose_your_y("bad")

In [ ]:
X = X.drop("reason_HomeImp",axis=1)

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X,y,test_size = 0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(100,20)).fit(X_train_scaled,y_train)
y_pred = mlp_model.predict(X_test_scaled)
print(classification_report(y_test,y_pred))

In [ ]:
mlpc_params = {
    'alpha':[0.1,0.01,0.001,0.005,0.0001,0.00001],
    'hidden_layer_sizes': [(10,10,10),(100,100),(100,100,100),(15,15),(30,30),(100)],
    'solver': ['lbfgs','adam','sgd'],
    'activation': ['relu','logistic','tanh','identity']  
}

In [ ]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc,mlpc_params,cv =10 , n_jobs=-1,verbose = 2,scoring='f1')
mlpc_cv_model.fit(X_train_scaled,y_train)

In [ ]:
mlpc_cv_model.best_params_

In [ ]:
mlpc_tuned = MLPClassifier(activation='relu',alpha=0.1,hidden_layer_sizes= (30,30),solver='lbfgs')

In [ ]:
mlpc_tuned.fit(X_train_scaled,y_train)
y_pred=mlpc_tuned.predict(X_test_scaled)
print(classification_report(y_test,y_pred))